In [9]:
import pathlib, pandas, numpy, random, string

## Load a file

Pandas can load many formats. I commonly use `read_csv` and `read_excel`.

`read_csv` allows the flexibilty of different separator (comma is default), header options and index (row names for those familar with R dataframes) options. 

In [23]:
dummy = pandas.read_csv('pandas/dummydataset.txt', sep = '\t')

## Explore the dataframe

A sanity check of your data is always a good idea.. is it the right size? has the file loaded as expected (incorrect delimiters or odd characters)? Is there any missing data? etc;

In [28]:
dummy.shape #274 rows and 3 columns

(274, 3)

In [30]:
dummy.head()

,Sample name,MLST,MLST_pcr
0,VGT9N5,-,NaN
1,U3IZ39,11,NaN
2,H7YRGD,213,NaN
3,PBGMLI,43,NaN
4,RM3OA4,7808,NaN


## Accessing different parts of the dataframe

A dataframe can be thought of as a large dictionary - containing many smaller identically structured dictionaries. Each of these smaller dictionaries are `key:value` pairs, where `key` is the column name and the `value` is the cell content. 

### Columns

Columns are essentially a list of values - these can be specified when you load the file if you like or modified later as a group (ie all the columns names at once) or individually

In [31]:
dummy.columns

Index(['Sample name', 'MLST', 'MLST_pcr'], dtype='object')

In [42]:
dummy.columns = ['Ioslate', 'MLST_A', 'MLST_B']

In [43]:
dummy.columns

Index(['Ioslate', 'MLST_A', 'MLST_B'], dtype='object')

In [44]:
dummy.columns[0] = ['Isolate']

TypeError: Index does not support mutable operations

In [45]:
dummy = dummy.rename(columns = {'Ioslate': 'Isolate'})

In [46]:
dummy.columns

Index(['Isolate', 'MLST_A', 'MLST_B'], dtype='object')

### Rows / Index

You can set the index of the dataframe to be a particular column if you like... Personally I don't see any advantage to this - since if you would like to subset based on values in a particular column this can be easily achieved with subsetting and slicing.

### Basic subsetting

Pandas dataframes can be subsetted and sliced in many many ways.. 

A single column - give the name of the column

In [49]:
dummy['Isolate'].head() # select all values in a column = this is called a series (it is a different type of object to a list)

0    VGT9N5
1    U3IZ39
2    H7YRGD
3    PBGMLI
4    RM3OA4
Name: Isolate, dtype: object

Multiple columns - Give a list of columns

In [55]:
dummy[['Isolate', 'MLST_A']].head()

,Isolate,MLST_A
0,VGT9N5,-
1,U3IZ39,11
2,H7YRGD,213
3,PBGMLI,43
4,RM3OA4,7808


Select the whole row based on a column value - give the name of the column and the value to select

In [50]:
dummy[dummy['Isolate'] == 'RM3OA4'] # returns a dataframe with only one row

,Isolate,MLST_A,MLST_B
4,RM3OA4,7808,NaN


Get the value of a cell, ie where an Isolate is of interest and the value of a single colum is needed use `.loc`. 

In [57]:
dummy.loc[dummy['Isolate'] == 'RM3OA4', 'MLST_A'] # returns a dataframe with only one row

4    7808
Name: MLST_A, dtype: object

### `.loc` vs `.iloc`

`.loc` is used when wanting to select based on the name of something, whereas `.iloc` is used if you want to find things based on the index number.

In [58]:
dummy.iloc[1, 1]

'11'

In [62]:
dummy.iloc[:10, :]

,Isolate,MLST_A,MLST_B
0,VGT9N5,-,NaN
1,U3IZ39,11,NaN
2,H7YRGD,213,NaN
3,PBGMLI,43,NaN
4,RM3OA4,7808,NaN
5,8VX6ZD,6058,NaN
6,T11YGO,1214,NaN
7,OZFKTG,10001,NaN
8,H5EC9P,32,NaN
9,Y2BKCQ,114,NaN


### Handling missing data

Often you will get a dataset that for some reason has missing information. It is important to understand how much is 'missing'. But also depending on what this means or how you need it to be represented for your particular purpose you may need to replace these missing values with something that is either meaningful to you or won't break any calculations that you want to make.

### Counting NaN

In a pandas dataframe, missing data is often represented as `NaN`. It is important to remember that this will occur if the particular 'cell' had NO DATA in it OR if the data/text there was recognised as an `NaN`. However, sometimes the missing data is represented as a string of some sort. 

In [63]:
dummy['MLST_B'].isna().sum()

214

In [64]:
dummy['MLST_A'].str.match('-').sum()

19

### Detect datatype of a column

`.isna()` will return `True` when the content of a cell is some representation of `NA`. If the original data was empty, this will be represented as a `numpy.NaN`.  However, characters such as an empty string ('') will be reported and `False`. So it can sometimes be helpful to know what types of data you are working with in order to figure out what your 'missing values' might look like.

In [76]:
dummy['MLST_B'].dtypes # O is object - which means that the column contains strings

dtype('O')

### Replacing data - missing values
If you have actual `NaN` then you can use the `.fillna()` function

In [77]:
dummy = dummy.fillna({'MLST_B':'No MLST'})

In [79]:
dummy.head()

,Isolate,MLST_A,MLST_B
0,VGT9N5,-,No MLST
1,U3IZ39,11,No MLST
2,H7YRGD,213,No MLST
3,PBGMLI,43,No MLST
4,RM3OA4,7808,No MLST


But if the 'missing value' is represented in some other way you can use either `replace` or `str.replace` (depending on the situation

In [80]:
dummy['MLST_A'] = dummy['MLST_A'].replace('-', 'No MLST')

In [82]:
dummy.head()

,Isolate,MLST_A,MLST_B
0,VGT9N5,No MLST,No MLST
1,U3IZ39,11,No MLST
2,H7YRGD,213,No MLST
3,PBGMLI,43,No MLST
4,RM3OA4,7808,No MLST


### Merging

You will often have a situation where there is a results table and you then get some sort of metadata in a separate table (hopefully with matching sample identifiers) that you will need to merge with your results. This is a relatively simple process in itself, the difficulties often arise in the cleaning up of the data so that there is some common key to work with.

In [93]:
metadata = pandas.read_csv('pandas/metadata.tab', sep = '\t')
metadata.head()

,Isolate,Contigs,bp
0,VGT9N5.ext,164,3366017
1,U3IZ39.ext,332,2111595
2,H7YRGD.ext,275,2212398
3,PBGMLI.ext,133,2095495
4,RM3OA4.ext,184,2182253


In [95]:
metadata.dtypes

Isolate    object
Contigs     int64
bp          int64
dtype: object

In [96]:
metadata['Contigs'].isna().sum()

0

In [97]:
metadata.shape

(274, 3)

### `apply`
If you want to edit a whole dataframe, a single column or even add a column dependent on some condition the `apply` function is a reliable workhorse

In [98]:
metadata['Isolate'] = metadata['Isolate'].apply(lambda x: x.strip('.ext'))

In [100]:
metadata.head()

,Isolate,Contigs,bp
0,VGT9N5,164,3366017
1,U3IZ39,332,2111595
2,H7YRGD,275,2212398
3,PBGMLI,133,2095495
4,RM3OA4,184,2182253


In [101]:
combined = pandas.merge(left = dummy, right = metadata, how = 'inner', on = 'Isolate')

In [103]:
combined.head()

,Isolate,MLST_A,MLST_B,Contigs,bp
0,VGT9N5,No MLST,No MLST,164,3366017
1,U3IZ39,11,No MLST,332,2111595
2,H7YRGD,213,No MLST,275,2212398
3,PBGMLI,43,No MLST,133,2095495
4,RM3OA4,7808,No MLST,184,2182253


### Saving

In [113]:
combined.to_csv('pandas/combined.txt', sep = '\t', index = False)

### Other things that I thought of ...

In [104]:
combined[combined['Contigs'] < 100]

,Isolate,MLST_A,MLST_B,Contigs,bp
136,LAKODL,No MLST,No MLST,35,2919188
137,J9U3SJ,No MLST,No MLST,99,4901360
151,2NUFAM,No MLST,No MLST,41,1725198
155,HBHH8E,No MLST,No MLST,68,1822955
175,ITYSO9,No MLST,No MLST,95,6965106
273,47Z47X,No MLST,No MLST,1,4411532


In [109]:
combined[(combined['Contigs'] > 300) & (combined['MLST_A'] != 'No MLST')]

,Isolate,MLST_A,MLST_B,Contigs,bp
1,U3IZ39,11,No MLST,332,2111595
6,T11YGO,1214,No MLST,1153,2174427
67,165CPD,1281,No MLST,576,2207227
123,F0L894,213,No MLST,303,2197685
134,4ZXGTL,184,184,569,2204001
135,CKTCXR,184,184,597,2207457
138,NG0HH6,11,11,401,2108817
139,DHKXVT,12004,new,700,2224354
140,66I49J,22,No MLST,1664,2233879
143,6LTLD9,22,No MLST,752,2290294


In [105]:
combined['Contigs'].mean()

253.0109489051095

In [112]:
combined_sorted = combined.sort_values(by = 'MLST_A')
combined_sorted.head()

,Isolate,MLST_A,MLST_B,Contigs,bp
7,OZFKTG,10001,No MLST,190,2156719
107,W7T1ZN,10498,No MLST,193,2179809
62,OIIU9N,1061,No MLST,222,2210977
163,26G9VX,11,11,383,2097793
164,GPDGT1,11,11,137,2101709
